In [ ]:
#| hide
from fh_saas.core import *

# 🚀 fh-saas

> **Production-ready multi-tenant SaaS toolkit for FastHTML applications.**

Build secure, scalable SaaS products without reinventing the wheel. `fh-saas` provides the backend infrastructure so you can focus on your product.

---

## ✨ What You Get Out of the Box

| Category | Features |
|----------|----------|
| 🏠 **Multi-Tenancy** | Isolated tenant databases, automatic routing, user-tenant memberships |
| 🔐 **Authentication** | OAuth integration (Google, GitHub), session management |
| 💳 **Billing** | Stripe subscriptions, plan tiers, usage tracking |
| 📧 **Communications** | Transactional emails via Resend, templated notifications |
| 🔌 **Integrations** | HTTP client with retries, GraphQL client with streaming pagination |
| ⚡ **Background Jobs** | Async task execution, job queue management |
| 📊 **Data Pipeline** | API → Polars transforms → Database sync |
| 📝 **Content** | Blog publishing, SEO & sitemaps, workflow automation |
| 🪝 **Webhooks** | Receive and process external webhooks securely |
| 📋 **Audit & Logging** | Security audit trails, configurable logging |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                      Your FastHTML App                          │
├─────────────────────────────────────────────────────────────────┤
│                         fh-saas                                 │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │ 🏠 Host DB   │  │ 🗄️ Tenant DBs │  │ 🔌 External  │          │
│  │  - Users     │  │  - App Data   │  │  - Stripe    │          │
│  │  - Tenants   │  │  - Isolated   │  │  - OAuth     │          │
│  │  - Billing   │  │  - Per-tenant │  │  - APIs      │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
└─────────────────────────────────────────────────────────────────┘
```

## ⚡ Quick Start

### 1. Install

```bash
pip install fh-saas
```

### 2. Configure Environment

```bash
# .env file
DB_TYPE=POSTGRESQL
DB_USER=postgres
DB_PASS=your_password
DB_HOST=localhost
DB_NAME=app_host

# Optional integrations
STRIPE_SECRET_KEY=sk_...
RESEND_API_KEY=re_...
GOOGLE_CLIENT_ID=...
GOOGLE_CLIENT_SECRET=...
```

### 3. Initialize Your App

```python
from fh_saas.db_host import HostDatabase, GlobalUser, TenantCatalog, Membership, gen_id, timestamp
from fh_saas.db_tenant import get_or_create_tenant_db
from fh_saas.utils_db import register_tables, create_indexes
from fh_saas.utils_log import configure_logging

# Configure logging (once at startup)
configure_logging()

# Connect to host database
host_db = HostDatabase.from_env()

# Create a new user
user = GlobalUser(
    id=gen_id(),
    email="founder@startup.com",
    oauth_id="google_abc123",
    created_at=timestamp()
)
host_db.global_users.insert(user)

# Create a tenant for their organization
tenant = TenantCatalog(
    id=gen_id(),
    name="Acme Corp",
    db_url="postgresql://...",
    created_at=timestamp()
)
host_db.tenant_catalogs.insert(tenant)

# Link user to tenant as owner
membership = Membership(
    id=gen_id(),
    user_id=user.id,
    tenant_id=tenant.id,
    profile_id=gen_id(),
    role="owner",
    created_at=timestamp()
)
host_db.memberships.insert(membership)
host_db.commit()
```

### 4. Work with Tenant Data

```python
# Get tenant's isolated database
tenant_db = get_or_create_tenant_db(tenant.id, tenant.name)

# Define your app's data models
class Project:
    id: str
    name: str
    status: str = "active"
    created_at: str

class Task:
    id: str
    project_id: str
    title: str
    completed: bool = False

# Register tables (creates if not exist)
tables = register_tables(tenant_db, [
    (Project, "projects", "id"),
    (Task, "tasks", "id"),
])

# Add indexes for performance
create_indexes(tenant_db, [
    ("tasks", ["project_id"], False, None),
    ("tasks", ["completed"], False, None),
])

# Use the tables
projects = tables["projects"]
projects.insert(Project(id=gen_id(), name="Launch MVP", created_at=timestamp()))
tenant_db.conn.commit()
```

---

## 📚 Documentation Guide

```{=html}
<table>
<tr><th>Section</th><th>What You'll Learn</th></tr>
<tr><td colspan="2"><strong>📦 Core</strong></td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/db_host.html">Multi-Tenant Setup</a></td><td>Host database, user management, tenant registry</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/db_tenant.html">Tenant Databases</a></td><td>Isolated databases per tenant, connection pooling</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_db.html">Table Management</a></td><td>Create tables & indexes from dataclasses</td></tr>
<tr><td colspan="2"><strong>🔌 Integrations</strong></td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_api.html">HTTP Client</a></td><td>REST APIs with retries, rate limiting, auth</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_graphql.html">GraphQL Client</a></td><td>Streaming pagination for large datasets</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_webhook.html">Webhooks</a></td><td>Receive & verify external webhooks</td></tr>
<tr><td colspan="2"><strong>⚙️ Data Pipeline</strong></td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_bgtsk.html">Background Tasks</a></td><td>Async job execution</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_polars_mapper.html">Data Transforms</a></td><td>JSON → Polars → Database pipeline</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_sync.html">API → DB Sync</a></td><td>End-to-end external data sync</td></tr>
<tr><td colspan="2"><strong>🛠️ Utilities</strong></td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_sql.html">SQL Helpers</a></td><td>Database type detection, query builders</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_log.html">Logging</a></td><td>Configurable logging for all modules</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_oauth.html">Authentication</a></td><td>OAuth flows (Google, GitHub)</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_email.html">Email Sending</a></td><td>Transactional emails via Resend</td></tr>
<tr><td colspan="2"><strong>📣 Content</strong></td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_blog.html">Blog Publishing</a></td><td>Markdown blog with frontmatter</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_seo.html">SEO & Sitemaps</a></td><td>Sitemap generation, meta tags</td></tr>
<tr><td><a href="https://abhisheksreesaila.github.io/fh-saas/utils_workflow.html">Workflow Engine</a></td><td>Multi-step automation</td></tr>
</table>
```

---

## 🛠️ Developer Guide

This project uses [nbdev](https://nbdev.fast.ai/) for literate programming. The source of truth is in the `nbs/` notebooks.

### Development Setup

```bash
# Clone and install in dev mode
git clone https://github.com/abhisheksreesaila/fh-saas.git
cd fh-saas
pip install -e .

# Make changes in nbs/ directory, then compile
nbdev_prepare
```

---

## 📦 Installation

```bash
# From PyPI (recommended)
pip install fh-saas

# From GitHub (latest)
pip install git+https://github.com/abhisheksreesaila/fh-saas.git
```

[GitHub](https://github.com/abhisheksreesaila/fh-saas) · [PyPI](https://pypi.org/project/fh-saas/) · [Documentation](https://abhisheksreesaila.github.io/fh-saas/)

**🤖 For AI Assistants:** [Download llms-ctx.txt](https://raw.githubusercontent.com/abhisheksreesaila/fh-saas/main/llms-ctx.txt) — Complete API documentation for LLMs

---

## 🤖 AI Assistant Context

**For AI coding assistants** (GitHub Copilot, Claude, ChatGPT, Cursor, etc.), download the complete API documentation:

**[📥 Download llms-ctx.txt](https://raw.githubusercontent.com/abhisheksreesaila/fh-saas/main/llms-ctx.txt)** — Complete fh-saas documentation in LLM-friendly format

### How to use:
1. Download the context file
2. Add to your AI assistant's instructions/knowledge base  
3. Get accurate code suggestions for fh-saas APIs

This file contains all module documentation, examples, and API signatures formatted for optimal LLM understanding.

---

## 🤝 Contributing

Contributions are welcome! Please check the [GitHub repository](https://github.com/abhisheksreesaila/fh-saas) for issues and discussions.

## 🤖 AI Assistant Context

**For AI coding assistants** (GitHub Copilot, Claude, ChatGPT, Cursor, etc.), download the complete API documentation:

### [📥 Download llms-ctx.txt](https://raw.githubusercontent.com/abhisheksreesaila/fh-saas/main/llms-ctx.txt)

**What's included:**
- Complete module documentation and API signatures
- Code examples and usage patterns
- Multi-tenant architecture patterns
- Authentication and security flows
- Data pipeline implementations

**How to use:**
1. Download the `llms-ctx.txt` file
2. Add it to your AI assistant's project context or instructions
3. Get accurate, context-aware code suggestions for fh-saas

This consolidated context file (~500KB) contains all documentation formatted for optimal LLM understanding, enabling your AI assistant to provide precise help with fh-saas APIs.